In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.1 MB/s eta 0:00:00


In [ ]:
!pip install pdf2image
!apt-get install poppler-utils -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 8.0 MB/s eta 0:00:00


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (986 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126441 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt-get install tesseract-ocr -y
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
import pdfplumber
ocr = False
with pdfplumber.open("/content/BOS_TY_EXTC_Syllabus.pdf") as pdf:
    first_page = pdf.pages[0]
    if not first_page:
        ocr = True

In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

pdf_path = "/content/BOS_TY_EXTC_Syllabus.pdf"
pages_text = []

if not ocr:  # digital PDF
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            pages_text.append({
                "page": i,
                "text": text,
                "source_file": pdf_path,
                "ocr": False
            })
else:  # scanned PDF
    pages = convert_from_path(pdf_path, dpi=300)
    for i, page in enumerate(pages, start=1):
        text = pytesseract.image_to_string(page, lang='eng+hin+ben+chi_sim')
        pages_text.append({
            "page": i,
            "text": text,
            "source_file": pdf_path,
            "ocr": True
        })


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5439339281ab10fadd485f73102d900c48ac5463d65eb36ddc86d91973aef342
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
#preprocessing the data
from langdetect import detect

for page in pages_text:
    if page["text"]:
        page["language"] = detect(page["text"])


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
chunk_size = 400   # tokens per chunk
overlap = 80       # tokens to overlap between chunks
chunks = []

for page in pages_text:
    tokens = tokenizer.encode(page["text"])
    i = 0
    while i < len(tokens):
        chunk_tokens = tokens[i:i+chunk_size]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append({
            "chunk_text": chunk_text,
            "page": page["page"],
            "source_file": page["source_file"],
            "language": page.get("language", "unknown")
        })
        i += (chunk_size - overlap)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
chunk_texts = [c["chunk_text"] for c in chunks]

embeddings = model.encode(chunk_texts, batch_size=64, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.1 MB/s eta 0:00:00


In [ ]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print(f"Stored {index.ntotal} chunks in index.")

Stored 96 chunks in index.


In [ ]:
def search(query, top_k=3):
    query_vec = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "rank": i+1,
            "text": chunks[idx]["chunk_text"],
            "page": chunks[idx]["page"],
            "source_file": chunks[idx]["source_file"],
            "distance": float(distances[0][i])
        })
    return results

# Example search
query = "What are the subjects in 5th semester?"
results = search(query, top_k=3)
for r in results:
    print(f"Rank {r['rank']} (Page {r['page']}): {r['text'][:200]}...\n")


Rank 1 (Page 25): [CLS] semester - vi 25 [SEP]...

Rank 2 (Page 32): [CLS] course outcomes : after the completion of course, the student should be able to 1. analyze the needs of various sections of society who can benefit from electronic interventions. 2. translate th...

Rank 3 (Page 47): semester - credits 2 scheme 3l : 0t : 0p course basics of antennas category : ec course outcomes : after the completion of course, the student should be able to 1. elaborate basic parameters of antenn...



In [ ]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.9 MB/s eta 0:00:00


In [ ]:
pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token="Hugging face token *")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token="Hugging face token *",
    device_map="auto",
    torch_dtype="auto"  # uses FP16 if GPU available
)


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

def search(query, top_k=3):
    query_vec = sentence_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "rank": i+1,
            "text": chunks[idx]["chunk_text"],
            "page": chunks[idx]["page"],
            "source_file": chunks[idx]["source_file"],
            "distance": float(distances[0][i])
        })
    return results

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def rag_answer(query, top_k=3, max_new_tokens=300):
    retrieved = search(query, top_k)
    context = "\n\n".join([r["text"] for r in retrieved])

    prompt = f"""
    You are a helpful assistant.
    Use the following context to answer the question.
    If the answer is not in the context, say you don't know.

    Context:
    {context}

    Question: {query}
    Answer:
    """

    response = llm_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    return response[0]["generated_text"]

Device set to use cpu


In [ ]:
query = "What are the subjects in 5th semester?"
answer = rag_answer(query, top_k=3)
print(answer)


    You are a helpful assistant.
    Use the following context to answer the question.
    If the answer is not in the context, say you don't know.

    Context:
    [CLS] semester - vi 25 [SEP]

[CLS] course outcomes : after the completion of course, the student should be able to 1. analyze the needs of various sections of society who can benefit from electronic interventions. 2. translate the need into an idea of electronic systems. 3. design and implement electronic systems using analog and digital components and sensors / transducers. module content hrs 1 analysis of an electronic aid / instrument / system 4 requirements analysis to gather inputs to describe needs of various sections of society. describe government of india schemes like - make in india, digital india, start up india, stand up india, smart cities mission, electronic manufacturing mission, software parks, hardware parks, facilities and support provided by various government agencies for these schemes. 2 design, fabr

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token="token*")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token="token*",
    device_map="auto",
    torch_dtype="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

def search(query, top_k=3):
    query_vec = sentence_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "rank": i+1,
            "text": chunks[idx]["chunk_text"],
            "page": chunks[idx]["page"],
            "source_file": chunks[idx]["source_file"],
            "distance": float(distances[0][i])
        })
    return results

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

query = "What are the subjects in 5th semester?"
search_results = search(query, top_k=3)
context = " ".join([r['text'] for r in search_results])


answer = qa_pipeline(f"Context: {context}\nQuestion: {query}\nAnswer:", max_new_tokens=150)
print(answer[0]['generated_text'])

Device set to use cpu


Context: [CLS] semester - vi 25 [SEP] [CLS] course outcomes : after the completion of course, the student should be able to 1. analyze the needs of various sections of society who can benefit from electronic interventions. 2. translate the need into an idea of electronic systems. 3. design and implement electronic systems using analog and digital components and sensors / transducers. module content hrs 1 analysis of an electronic aid / instrument / system 4 requirements analysis to gather inputs to describe needs of various sections of society. describe government of india schemes like - make in india, digital india, start up india, stand up india, smart cities mission, electronic manufacturing mission, software parks, hardware parks, facilities and support provided by various government agencies for these schemes. 2 design, fabricate and test an electronic system 5 describe un sustainable development goals. design, fabricate, implement and test an electronic system with both analog an